<a href="https://colab.research.google.com/github/wmmandela/Mandela/blob/main/Hate_Speech_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset
df = pd.read_csv("HateSpeech_Kenya.csv")

# Define a stopwords list manually
custom_stopwords = set(["i", "me", "my", "we", "our", "you", "your", "he", "she", "it", "they", "them", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "can", "will", "just", "don", "should", "now"])

# Function to clean text
def preprocess_text(text):
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'\W', ' ', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    tokens = text.split()  # Tokenization using split
    tokens = [word for word in tokens if word not in custom_stopwords]  # Remove stopwords
    return ' '.join(tokens)

# Apply preprocessing
df['Clean_Tweet'] = df['Tweet'].astype(str).apply(preprocess_text)

# Prepare dataset for model training
X = df['Clean_Tweet']
y = df['Class']

# Split into training and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert text data to numerical format using TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Train a Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# Predictions and evaluation
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, labels=[0, 1, 2], target_names=['Neither', 'Offensive', 'Hate Speech'])


# Print results
print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", report)


Accuracy: 0.7486
Classification Report:
               precision    recall  f1-score   support

     Neither       0.75      1.00      0.86      7166
   Offensive       0.52      0.03      0.06      1806
 Hate Speech       0.55      0.01      0.02       644

    accuracy                           0.75      9616
   macro avg       0.61      0.35      0.31      9616
weighted avg       0.69      0.75      0.65      9616

